<a href="https://colab.research.google.com/github/ridaIndah/Proyek-Mandiri/blob/main/simpleTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf

In [ ]:
import os
import re
import pandas as pd
import torch
import tensorflow as tf

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Proyek Mandiri/data/DDNew2.csv')

In [ ]:
#Reducing the training sample for fast execution
df = df.sample(frac = 0.2)

In [ ]:
df.head()

,utterance,emotion
3349,How much antifreeze did you put in your new ca...,0
91564,"Sure , anything you say .",4
74022,Roast beef .,0
96424,"Good evening , Do you have a reservation ?",0
8013,How about going for swim around 10,4


In [ ]:
#splitting the training set in to training and validation sets
from sklearn.model_selection import train_test_split

train, val =  train_test_split(df, test_size = 0.2, random_state = 120)

In [ ]:
train.shape

(16476, 2)

# **Begin with Transformer**

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 225kB 8.8MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
     |████████████████████████████████| 3.3MB 27.3MB/s 
     |████████████████████████████████| 1.8MB 39.1MB/s 
     |████████████████████████████████| 7.8MB 47.6MB/s 
     |████████████████████████████████| 122kB 52.3MB/s 
     |████████████████████████████████| 2.5MB 40.0MB/s 
     |████████████████████████████████| 245kB 48.3MB/s 
     |████████████████████████████████| 1.2MB 46.4MB/s 
     |████████████████████████████████| 133kB 57.8MB/s 
     |████████████████████████████████| 174kB 40.0MB/s 
     |████████████████████████████████| 102kB 14.0MB/s 
     |████████████████████████████████| 112kB 51.7MB/s 
     |████████████████████████████████| 4.2MB 44.5MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
     |████████████████████████████████| 901kB 39.6MB/s 
     |████████████████████████████████| 245kB 51.3MB/s

In [ ]:
from simpletransformers.classification import ClassificationModel

In [ ]:
model = ClassificationModel('roberta', 'roberta-base', num_labels=7)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.

In [ ]:
import torch
classifier = torch.nn.Sequential(torch.nn.BatchNorm1d(512), torch.nn.Linear(512, 7))

In [ ]:
#training the classifier
model.train_model(train)
#save model
model.save('/content/drive/MyDrive/Proyek Mandiri')   

In [ ]:
scores1, model_outputs, wrong_predictions = model.eval_model(val)

print("eval_loss", scores1)



/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1384: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



eval_loss {'mcc': 0.42642358385905643, 'eval_loss': 0.5477073002787469}


In [ ]:
#Evaluating With F1 Score & Accuracy

from sklearn.metrics import f1_score, accuracy_score
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

scores2, model_outputs, wrong_predictions = model.eval_model(val, f1=f1_multiclass, acc=accuracy_score)

print(scores2)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1384: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



{'mcc': 0.42642358385905643, 'f1': 0.8519417475728155, 'acc': 0.8519417475728155, 'eval_loss': 0.5477073002787469}
